# Part 1

Importing libraries

In [45]:
!pip install geocoder
!pip install folium

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
    100% |████████████████████████████████| 92kB 18.5MB/s ta 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [77]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import geocoder
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

Importing webpage

In [8]:
wiki_link = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(wiki_link).text

In [9]:
soup = BeautifulSoup(page, 'lxml')
my_table = soup.find('table',{'class':'wikitable sortable'})

extracting table

In [10]:
data =[]
rows = my_table.find_all('tr')

for tr in rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if row:
        data.append(row)

Cleaning DF

In [11]:
df = pd.DataFrame(data, columns = ['Postcode', 'Borough', 'Neighbourhood'])
df = df[df.Borough !='Not assigned']
df['Neighbourhood'].replace(r"\n","", regex=True, inplace=True)
df['Neighbourhood'].replace(to_replace = 'Not assigned', value = df['Borough'], inplace = True)
df.head(5)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [34]:
df1 = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df1 = df1.sort_values(by = 'Postcode')
df1.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
df1.shape

(103, 3)

# Part 2

In [38]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')
df_geo.rename(columns={'Postal Code': 'Postcode'}, inplace = True)

df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [152]:
df_tor = pd.merge(df1, df_geo, on='Postcode')
df_tor.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [42]:
df_tor.shape

(103, 5)

# Part 3

Preparing DF for different Borough clusters




In [162]:
kclusters = df_tor['Borough'].nunique()
df_b = pd.DataFrame(df_tor['Borough'].unique(), columns =['Borough'])
df_b['Cluster']= np.arange(1,kclusters+1)
df_b.head()

,Borough,Cluster
0,Scarborough,1
1,North York,2
2,East York,3
3,East Toronto,4
4,Central Toronto,5


In [159]:
df_map = pd.merge(df_tor, df_b, on='Borough')

Drawing map

In [166]:
# set number of clusters
kclusters = df_map['Borough'].nunique()

address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow

# markers_colors = []
for lat, lon, neighbourhood, borough, cluster in zip(df_map['Latitude'], df_map['Longitude'], df_map['Neighbourhood'], df_map['Borough'], df_map['Cluster']):
    label = '{}, {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_tor)
       
map_tor
